# Problema 3
$$
\begin{array}{ll}
\operatorname{minimizar} & \frac{1}{2} \vec{\boldsymbol{x}}^{\top} G \vec{\boldsymbol{x}}+\vec{\boldsymbol{c}}^{\top} \vec{\boldsymbol{x}} \\
\text { sujeto a } & A \vec{x}=\vec{\boldsymbol{b}} \\
& x_{i} \geq \ell_{i} \quad \text { si } \ell_{i} \text { es finito, } \\
& x_{i} \leq u_{i} \quad \text { si } u_{i} \text { es finito. }
\end{array}
$$

- Definimos $J \subset I$ como sigue:

- Para $x_{j} \geq \ell_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|\ell_{j}\right|, 1\right\} \Longrightarrow j \in J$

- Para $x_{j} \leq u_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|u_{j}\right|, 1\right\} \Longrightarrow j \in J$

Donde $\varepsilon_{m}$ es el épsilon de la máquina `eps(Float64)`

In [1]:
include("Solvers.jl")
using .Solvers, LinearAlgebra, MAT, .Solvers.Utils

In [2]:
problem = matread("../data/lpi_woodinfe.mat")["Problem"]

Dict{String, Any} with 11 entries:
  "A"      => …
  "aux"    => Dict{String, Any}("c"=>[10.0; 11.0; … ; 15.0; 15.0], "hi"=>[100.0…
  "b"      => [0.0; 0.0; … ; 0.0; 0.0]
  "date"   => "1989"
  "name"   => "LPnetlib/lpi_woodinfe"
  "kind"   => "linear programming problem"
  "author" => "H. Greenberg"
  "id"     => 731.0
  "notes"  => ["An infeasible Netlib LP problem, in lp/infeas.  For more inform…
  "title"  => "Netlib LP problem woodinfe: minimize c'*x, where Ax=b, lo<=x<=hi"
  "ed"     => "J. Chinneck"

In [3]:
b = problem["b"] 
c = problem["aux"]["c"][:] # El [:] es para interpretar como vector
l = problem["aux"]["lo"][:] # El [:] es para interpretar como vector
u = problem["aux"]["hi"][:] # El [:] es para interpretar como vector
n_eq = length(b)
A_eq = problem["A"]
G = I(length(c))

# Como all(isfinite.(l)) == true, se complen todas las cotas inferiores
A = [A_eq; -I(length(l))]
b = [b; -l]

# Imponiendo cotas superiores
# Concatenando las filas I_j de una identidad tal que u_j < Inf
mask = isfinite.(u)
A = [A; I(length(u))[mask, :]]
# Igual cambiando b para que dimensiones coincidan
b = [b; u[mask]]
b = b[:]

138-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   ⋮
 200.0
 100.0
  25.0
  10.0
  50.0
  40.0
  20.0
   5.0
  30.0

In [4]:
# Definiendo el conjunto inicial de igualdades
# Obtenemos x_0 con linprog (pero ya sabemos que va a ser vec(0))
x_0 = linprog(A, b, n_eq)

# Shorthand para epsilon de maquina
εₘ = eps(Float64)

# Obteniendo los masks que indican si se usa la restricción
# Recordamos que para las cotas inferiores g_i(x) = l_i - x_i & para las sup g_i(x) = x_i - u_i
J_l = abs.(l - x_0) .<= 8 * εₘ *  max.(abs.(l), ones(length(l)))

# Ahora para las superiores, recordando quitar los renglones donde u_j no es finito (por size de A).
J_u = abs.(x_0 - u) .<= 8 * εₘ *  max.(abs.(u), ones(length(u)))
J_u = J_u[isfinite.(u)]

J = [J_l; J_u]

# Concatenando con un índice aleatorio
R_index = rand(findall(J), 1)
notJ = falses(size(A, 1) - n_eq)
notJ[R_index] .= true
W_0 = [trues(n_eq); notJ]

138-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [5]:
println(W_0)

Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Nótese que el número de filas de W_0 coincide con el de A, y el tamaño de b

In [6]:
@assert size(A, 1) == length(W_0)
@assert size(A, 1) == length(b)

In [7]:
iters, x_star, q_star, μ = activeSetMethod(G, c, A, b, n_eq, copy(W_0))

α = -0.0, j = 38 

Rama 1. ||d_k|| = 158.6, q(x) = 90250.0, α=-0.0k = 0
α = -0.0, j = 39 

Rama 1. ||d_k|| = 158.8, q(x) = 90250.0, α=-0.0k = 1
α = -0.0, j = 40 

Rama 1. ||d_k|| = 159.5, q(x) = 90250.0, α=-0.0k = 2
α = -0.0, j = 42 

Rama 1. ||d_k|| = 160.2, q(x) = 90250.0, α=-0.0k = 3
α = -0.0, j = 43 

Rama 1. ||d_k|| = 160.9, q(x) = 90250.0, α=-0.0k = 4
α = -0.0, j = 77 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 5
α = -0.0, j = 79 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 6
α = -0.0, j = 85 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 7
α = -0.0, j = 88 

Rama 1. ||d_k|| = 162.1, q(x) = 90250.0, α=-0.0k = 8
α = -0.0, j = 89 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 9
α = -0.0, j = 90 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 10
α = -0.0, j = 92 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 11
α = -0.0, j = 95 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 12
α = -0.0, j = 91 

Rama 1. ||d_k|| = 162.9, q(x) = 90250.0, α


α = 2.353, j = 96 

Rama 1. ||d_k|| = 4.25, q(x) = 67370.0, α=2.353
Rama 2. 
α = 1.429, j = 110 

Rama 1. ||d_k|| = 3.5, q(x) = 67350.0, α=1.429
Rama 2. 
α = 2.27, j = 86 

Rama 1. ||d_k|| = 5.727, q(x) = 67300.0, α=2.27
Rama 2. 
α = 2.077, j = 117 

Rama 1. ||d_k|| = 4.816, q(x) = 67270.0, α=2.077
Rama 2. 
α = 3.327, j = 77 

Rama 1. ||d_k|| = 5.524, q(x) = 67240.0, α=3.327
Rama 2. 
α = 1.595, j = 101 

Rama 1. ||d_k|| = 3.135, q(x) = 67220.0, α=1.595
Rama 2. 
α = 4.201, j = 104 

Rama 1. ||d_k|| = 1.19, q(x) = 67220.0, α=4.201
Rama 2. 
α = 3.165, j = 101 

Rama 1. ||d_k|| = 1.279, q(x) = 67220.0, α=3.165
Rama 2. 


α = 9.982, j = 113 

Rama 1. ||d_k|| = 0.9649, q(x) = 67220.0, α=9.982
Rama 2. j = 1, μ=0.0 


89-element Vector{Float64}:
 51.47
 47.0
 49.53
 50.0
 10.0
 49.47
 43.0
 49.53
 25.0
 10.0
  ⋮
 15.33
  4.702
  6.364
  6.061
 30.0
  3.734e-13
  6.455
  9.264e-14
  9.333

Concluyó método del conjunto activo en 77 iteraciones
El punto de paro fue:


In [8]:
print(round.(x_star, sigdigits=4))

[51.47, 47.0, 49.53, 50.0, 10.0, 49.47, 43.0, 49.53, 25.0, 10.0, 35.0, 44.26, 34.46, 20.0, 5.0, 26.27, 30.0, 65.0, 100.0, 100.0, 90.0, 50.0, 10.0, 20.0, 25.0, 10.0, 15.0, 0.0, 50.0, 40.0, 20.0, 5.0, 15.0, 20.0, 25.0, 30.0, 20.0, 0.0, 42.69, 20.72, 33.06, 6.07, 15.19, 2.479, 10.0, 15.0, 43.8, 15.64, 25.32, 33.54, 3.636, 7.87, 12.67, 5.727, 1.279, 39.27, -1.482e-14, 11.28, 9.667, 37.64, 7.182, 7.479, 9.667, 22.14, 2.818, 1.242, 10.67, 15.96, 3.9, 15.33, 1.1, 18.7, 5.956e-14, 0.9649, 4.364, 9.506, 10.33, 4.333, 17.82, 10.71, 15.33, 4.702, 6.364, 6.061, 30.0, 3.734e-13, 6.455, 9.264e-14, 9.333]

In [9]:
q_star

108291.05263157882

## Benchmarks

In [14]:
using BenchmarkTools
using Plots
pgfplotsx(); theme(:ggplot2)

# Corriendo el benchmark de acuerdo a las best practices
bench = @benchmark activeSetMethod($G, $c, $A, $b, $n_eq, W_k) setup=(W_k = copy(W_0))

α = -0.0, j = 38 

Rama 1. ||d_k|| = 158.6, q(x) = 90250.0, α=-0.0k = 0
α = -0.0, j = 39 

Rama 1. ||d_k|| = 158.8, q(x) = 90250.0, α=-0.0k = 1
α = -0.0, j = 40 

Rama 1. ||d_k|| = 159.5, q(x) = 90250.0, α=-0.0k = 2
α = -0.0, j = 42 

Rama 1. ||d_k|| = 160.2, q(x) = 90250.0, α=-0.0k = 3
α = -0.0, j = 43 

Rama 1. ||d_k|| = 160.9, q(x) = 90250.0, α=-0.0k = 4
α = -0.0, j = 77 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 5
α = -0.0, j = 79 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 6
α = -0.0, j = 85 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 7
α = -0.0, j = 88 

Rama 1. ||d_k|| = 162.1, q(x) = 90250.0, α=-0.0k = 8
α = -0.0, j = 89 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 9
α = -0.0, j = 90 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 10
α = -0.0, j = 92 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 11
α = -0.0, j = 95 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 12
α = -0.0, j = 91 

Rama 1. ||d_k|| = 162.9, q(x) = 90250.0, α


α = 1.869, j = 107 

Rama 1. ||d_k|| = 26.75, q(x) = 74050.0, α=1.869
Rama 2. 
α = 2.667, j = 38 

Rama 1. ||d_k|| = 43.12, q(x) = 72560.0, α=2.667
Rama 2. 
α = 2.667, j = 99 

Rama 1. ||d_k|| = 31.16, q(x) = 71300.0, α=2.667
Rama 2. 
α = 2.593, j = 36 

Rama 1. ||d_k|| = 38.57, q(x) = 70130.0, α=2.593
Rama 2. 
α = 3.558, j = 76 

Rama 1. ||d_k|| = 19.4, q(x) = 69640.0, α=3.558
Rama 2. 
α = 2.483, j = 131 

Rama 1. ||d_k|| = 40.27, q(x) = 68390.0, α=2.483
Rama 2. 
α = 1.5, j = 112 

Rama 1. ||d_k|| = 10.0, q(x) = 68190.0, α=1.5
Rama 2. 
α = 3.048, j = 107 

Rama 1. ||d_k|| = 15.72, q(x) = 67900.0, α=3.048
Rama 2. 
α = 1.875, j = 124 

Rama 1. ||d_k|| = 10.67, q(x) = 67730.0, α=1.875
Rama 2. 
α = 1.712, j = 134 

Rama 1. ||d_k|| = 8.0, q(x) = 67620.0, α=1.712
Rama 2. 
α = 3.352, j = 78 

Rama 1. ||d_k|| = 12.98, q(x) = 67460.0, α=3.352
Rama 2. 
α = 2.526, j = 93 

Rama 1. ||d_k|| = 5.937, q(x) = 67410.0, α=2.526
Rama 2. 
α = 2.353, j = 96 

Rama 1. ||d_k|| = 4.25, q(x) = 67370.0, α=2.3


Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=-0.0k = 22
α = -0.0, j = 118 

Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=-0.0k = 23
α = -0.0, j = 119 

Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=-0.0k = 24
α = -0.0, j = 121 

Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=-0.0k = 25
α = -0.0, j = 123 

Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=-0.0k = 26
α = 0.0, j = 54 

Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=0.0k = 27
α = 0.0, j = 55 

Rama 1. ||d_k|| = 162.6, q(x) = 90250.0, α=0.0k = 28
α = 0.0, j = 56 

Rama 1. ||d_k|| = 132.7, q(x) = 90250.0, α=0.0k = 29
α = 0.0, j = 57 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 30
α = 0.0, j = 58 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 31
α = 0.0, j = 59 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 32
α = 0.0, j = 60 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 33
α = 0.0, j = 61 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 34
α = 0.0, j = 62 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 35
α = 0.0, j


α = 1.869, j = 107 

Rama 1. ||d_k|| = 26.75, q(x) = 74050.0, α=1.869
Rama 2. 
α = 2.667, j = 38 

Rama 1. ||d_k|| = 43.12, q(x) = 72560.0, α=2.667
Rama 2. 
α = 2.667, j = 99 

Rama 1. ||d_k|| = 31.16, q(x) = 71300.0, α=2.667
Rama 2. 
α = 2.593, j = 36 

Rama 1. ||d_k|| = 38.57, q(x) = 70130.0, α=2.593
Rama 2. 
α = 3.558, j = 76 

Rama 1. ||d_k|| = 19.4, q(x) = 69640.0, α=3.558
Rama 2. 
α = 2.483, j = 131 

Rama 1. ||d_k|| = 40.27, q(x) = 68390.0, α=2.483
Rama 2. 
α = 1.5, j = 112 

Rama 1. ||d_k|| = 10.0, q(x) = 68190.0, α=1.5
Rama 2. 
α = 3.048, j = 107 

Rama 1. ||d_k|| = 15.72, q(x) = 67900.0, α=3.048
Rama 2. 
α = 1.875, j = 124 

Rama 1. ||d_k|| = 10.67, q(x) = 67730.0, α=1.875
Rama 2. 
α = 1.712, j = 134 

Rama 1. ||d_k|| = 8.0, q(x) = 67620.0, α=1.712
Rama 2. 
α = 3.352, j = 78 

Rama 1. ||d_k|| = 12.98, q(x) = 67460.0, α=3.352
Rama 2. 
α = 2.526, j = 93 

Rama 1. ||d_k|| = 5.937, q(x) = 67410.0, α=2.526
Rama 2. 
α = 2.353, j = 96 

Rama 1. ||d_k|| = 4.25, q(x) = 67370.0, α=2.3

α = -0.0, j = 38 

Rama 1. ||d_k|| = 158.6, q(x) = 90250.0, α=-0.0k = 0
α = -0.0, j = 39 

Rama 1. ||d_k|| = 158.8, q(x) = 90250.0, α=-0.0k = 1
α = -0.0, j = 40 

Rama 1. ||d_k|| = 159.5, q(x) = 90250.0, α=-0.0k = 2
α = -0.0, j = 42 


...


Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 40
α = 0.0, j = 69 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 41
α = 0.0, j = 70 

Rama 1. ||d_k|| = 94.16, q(x) = 90250.0, α=0.0k = 42
α = 0.0, j = 71 

Rama 1. ||d_k|| = 87.21, q(x) = 90250.0, α=0.0k = 43
α = 0.0, j = 72 

Rama 1. ||d_k|| = 59.18, q(x) = 90250.0, α=0.0k = 44
α = 0.0, j = 125 

Rama 1. ||d_k|| = 26.0, q(x) = 90250.0, α=0.0k = 45
α = 0.0, j = 132 

Rama 1. ||d_k|| = 15.75, q(x) = 90250.0, α=0.0k = 46
α = 0.0, j = 133 

Rama 1. ||d_k|| = 15.75, q(x) = 90250.0, α=0.0k = 47
α = 0.0, j = 136 

Rama 1. ||d_k|| = 15.75, q(x) = 90250.0, α=0.0k = 48
α = 0.0, j = 137 

Rama 1. ||d_k|| = 15.75, q(x) = 90250.0, α=0.0k = 49
α = 0.0, j = 138 

Rama 1. ||d_k|| = 15.75, q(x) = 90250.0, α=0.0k = 50
α = 3.832, j = 75 

Rama 1. ||d_k|| = 9.6, q(x) = 90080.0, α=3.832
Rama 2. 
α = 0.8197, j = 128 

Rama 1. ||d_k|| = 61.0, q(x) = 82880.0, α=0.8197k = 52
Rama 2. 
α = 1.274, j = 78 

Rama 1. ||d_k|| = 39.25, q(x) = 79790.0, α=1.274



α = 0.0, j = 58 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 31
α = 0.0, j = 59 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 32
α = 0.0, j = 60 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 33
α = 0.0, j = 61 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 34
α = 0.0, j = 62 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 35
α = 0.0, j = 64 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 36
α = 0.0, j = 65 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 37
α = 0.0, j = 66 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 38
α = 0.0, j = 67 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 39
α = 0.0, j = 68 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 40
α = 0.0, j = 69 

Rama 1. ||d_k|| = 105.2, q(x) = 90250.0, α=0.0k = 41
α = 0.0, j = 70 

Rama 1. ||d_k|| = 94.16, q(x) = 90250.0, α=0.0k = 42
α = 0.0, j = 71 

Rama 1. ||d_k|| = 87.21, q(x) = 90250.0, α=0.0k = 43
α = 0.0, j = 72 

Rama 1. ||d_k|| = 59.18, q(x) = 90250.0, α=0.0k = 44
α = 0

Excessive output truncated after 524304 bytes.


Rama 2. 
α = 3.165, j = 101 

Rama 1. ||d_k|| = 1.279, q(x) = 67220.0, α=3.165
Rama 2. 
α = 9.982, j = 113 

Rama 1. ||d_k|| = 0.9649, q(x) = 67220.0, α=9.982
Rama 2. j = 1, μ=0.0 
α = -0.0, j = 38 

Rama 1. ||d_k|| = 158.6, q(x) = 90250.0, α=-0.0k = 0
α = -0.0, j = 39 

Rama 1. ||d_k|| = 158.8, q(x) = 90250.0, α=-0.0k = 1
α = -0.0, j = 40 

Rama 1. ||d_k|| = 159.5, q(x) = 90250.0, α=-0.0k = 2
α = -0.0, j = 42 

Rama 1. ||d_k|| = 160.2, q(x) = 90250.0, α=-0.0k = 3
α = -0.0, j = 43 

Rama 1. ||d_k|| = 160.9, q(x) = 90250.0, α=-0.0k = 4
α = -0.0, j = 77 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 5
α = -0.0, j = 79 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 6
α = -0.0, j = 85 

Rama 1. ||d_k|| = 161.8, q(x) = 90250.0, α=-0.0k = 7
α = -0.0, j = 88 

Rama 1. ||d_k|| = 162.1, q(x) = 90250.0, α=-0.0k = 8
α = -0.0, j = 89 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 9
α = -0.0, j = 90 

Rama 1. ||d_k|| = 162.8, q(x) = 90250.0, α=-0.0k = 10
α = -0.0, j = 92 

Rama 1

BenchmarkTools.Trial: 42 samples with 1 evaluation.
 Range (min … max):  102.039 ms … 196.765 ms  ┊ GC (min … max):  0.00% … 8.27%
 Time  (median):     120.942 ms               ┊ GC (median):    13.35%
 Time  (mean ± σ):   121.101 ms ±  18.822 ms  ┊ GC (mean ± σ):   8.36% ± 6.81%

  █▂  ▂      █▅▂▅ ▂                                              
  ██▅▅██▁▁▁▁▅████▅█▅▁▅▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▅ ▁
  102 ms           Histogram: frequency by time          197 ms <

 Memory estimate: 45.41 MiB, allocs estimate: 36556.

In [15]:
bench

BenchmarkTools.Trial: 42 samples with 1 evaluation.
 Range (min … max):  102.039 ms … 196.765 ms  ┊ GC (min … max):  0.00% … 8.27%
 Time  (median):     120.942 ms               ┊ GC (median):    13.35%
 Time  (mean ± σ):   121.101 ms ±  18.822 ms  ┊ GC (mean ± σ):   8.36% ± 6.81%

  █▂  ▂      █▅▂▅ ▂                                              
  ██▅▅██▁▁▁▁▅████▅█▅▁▅▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▅ ▁
  102 ms           Histogram: frequency by time          197 ms <

 Memory estimate: 45.41 MiB, allocs estimate: 36556.

In [16]:
ts = bench.times
h = histogram(ts, xlabel="Tiempo (s)", legend = :none, framestyle = :box);

Plot{Plots.PGFPlotsXBackend() n=1}

In [17]:
# savefig(h, "../docs/histograma.tex")